In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

import numpy as np
import re
import torch
import json
from masked_dust3r.scripts.utils.math import *

DATA_PATH = "/dust3r/masked_dust3r/data/jackal_training_data_0"

with open(os.path.join(DATA_PATH, "transforms.json"), "r") as f:
    transforms = json.load(f)

device = "cpu"

/dust3r


In [2]:
store_frames = []

for idx, f in enumerate(transforms["frames"]):
    if idx == len(transforms["frames"])-1:
        break
    p0 = np.array(transforms["frames"][idx]["transform_matrix"])
    p1 = np.array(transforms["frames"][idx+1]["transform_matrix"])

    cnt_0 = re.split("\.|/", transforms["frames"][idx]["file_path"])[-2]
    cnt_0 = int(cnt_0)
    cnt_1 = re.split("\.|/", transforms["frames"][idx+1]["file_path"])[-2]
    cnt_1 = int(cnt_1)

    poses = interpolate_pose(p0, p1, cnt_1 - cnt_0, device=device)

    img_path_arr = re.split("\.|/", transforms["frames"][idx]["file_path"])
    mask_path_arr = re.split("\.|/", transforms["frames"][idx]["mask_path"])

    for i in range(cnt_0, cnt_1):
        new_frame = {}
        new_frame["transform_matrix"] = poses[i-cnt_0].tolist()

        img_path_arr[-2] = str(i)
        mask_path_arr[-2] = str(i)

        new_frame["file_path"] = img_path_arr[0] + "/" + str(i) + "." + ".".join(img_path_arr[2:])
        new_frame["mask_path"] = mask_path_arr[0] + "/" + str(i) + "." + ".".join(mask_path_arr[2:])

        store_frames.append(new_frame)

print(len(store_frames))

transforms["frames"] = store_frames

with open(os.path.join(DATA_PATH, "transforms.json"), "w") as f:
    json.dump(transforms, f, indent=4)


248
